In [2]:
#调用torch库
import torch
import torchvision
import torchvision.transforms as transforms

import torch.nn as nn
import torch.nn.functional as F
#调用numpy库
import numpy as np
#调用matplotlib库
import matplotlib.pyplot as plt
#调用random库
import random

按要求生成二分类模型训练数据与验证数据

In [3]:
n_data = torch.zeros(10000, 200)
X1 = torch.normal(mean=n_data + 1, std=n_data + 0.01)
Y1 = torch.zeros(10000, 1)

X2 = torch.normal(mean=n_data - 1, std=n_data + 0.01)
Y2 = torch.ones(10000, 1)

x_train = torch.cat([X1[:7000], X2[:7000]], dim=0)
x_test = torch.cat([X1[7000:], X2[7000:]], dim=0)
y_train = torch.cat([Y1[:7000], Y2[:7000]], dim=0)
y_test = torch.cat([Y1[7000:], Y2[7000:]], dim=0)

In [4]:
print(x_train.shape, x_test.shape)
print(y_train.shape, y_test.shape)

torch.Size([14000, 200]) torch.Size([6000, 200])
torch.Size([14000, 1]) torch.Size([6000, 1])


In [5]:
#使用课上给出的数据读取函数
def data_iter(batch_size, features, labels):
    num_examples = len(features)
    indices = list(range(num_examples))
    random.shuffle(indices) # 样本的读取顺序是随机的
    for i in range(0, num_examples, batch_size):
        j = torch.LongTensor(indices[i: min(i + batch_size, num_examples)]) # 最后一次可能不足一个batch
        yield features.index_select(0, j), labels.index_select(0, j)

手动实现的前馈神经网络

In [6]:
# 使用课上给出的优化函数进行参数的学习优化
def sgd(params, lr, batch_size):
    for param in params:
        param.data -= lr * param.grad / batch_size

In [7]:
class Feedforward_twoclass:
    def __init__(self, feature_dim, opt, device, hidden=100):
        self.w_in = torch.tensor(np.random.normal(0, 0.01, (feature_dim, hidden)), dtype=torch.float32).to(device)
        self.b_in = torch.zeros(hidden, dtype=torch.float32).to(device)
        self.w_in.requires_grad_(requires_grad=True)
        self.b_in.requires_grad_(requires_grad=True) 

        self.w_out = torch.tensor(np.random.normal(0, 0.01, (hidden, 1)), dtype=torch.float32).to(device)
        self.b_out = torch.zeros(1, dtype=torch.float32).to(device)
        self.w_out.requires_grad_(requires_grad=True)
        self.b_out.requires_grad_(requires_grad=True) 

        self.opt = opt

    def forward(self, x):
        x = torch.matmul(x, self.w_in) + self.b_in
        y_hat = torch.matmul(x, self.w_out) + self.b_out
        return y_hat

    def learn(self, lr, bs):
        self.opt([self.w_in, self.b_in, self.w_out, self.b_out], lr, bs)

    def reset_grad(self):
        self.w_in.grad.data.zero_()
        self.b_in.grad.data.zero_()
        self.w_out.grad.data.zero_()
        self.b_out.grad.data.zero_()

In [8]:
epoch_num = 30
lr = 0.03
feature_dim = 200
device = 'cuda:0'
opt = sgd
loss = torch.nn.BCELoss()
batch_size = 128
model = Feedforward_twoclass(feature_dim, sgd, device)
train_loss_list = []
test_loss_list = []

: 

: 

In [ ]:
for epoch in range(epoch_num):
    for x, y in data_iter(batch_size, x_train, y_train):
        print(device)
        x, y = x.to(device), y.to(device)
        y_hat = model.forward(x).squeeze(1)
        y = y.squeeze(1)
        print(y_hat.device, y.device)
        l = loss(y_hat, y).sum()
        l.backward()
        model.learn(lr, batch_size)
        model.reset_grad()
        train_loss_list.append(l.item())
        print(l.item())
    
    # with torch.no_grad():
    #     total_loss = 0
    #     counter = 0
    #     for x, y in data_iter(batch_size, x_train, y_train):
    #         test_x, test_y = x.to(device), y.to(device)
    #         test_pred = model.forward(test_x)

    #         l = loss(test_pred, test_y).sum()
    #         total_loss += l
    #         counter += 1
    #     test_loss_list.append(total_loss.item())
    #     print('epoch: %d, total loss: %f'%(epoch + 1, total_loss / (counter+1)))

NameError: name 'epoch_num' is not defined